## Подсчет профиля города

### Содержание:
* Подгрузка выборки
* Подсчет количества по годам
* Подсчет лидеров заявителей города
* [Подсчет количества по областям с fractional count](#second-bullet)
* Подсчет заявителей по областям с $ИТС \geq 1$

In [1]:
import pandas as pd
from glob import glob
from collections import defaultdict
import numpy as np

Подгрузим загруженную с patstat выборку

In [2]:
CITY_NAME = 'роли'

In [3]:
df = pd.DataFrame([])
for indx, path in enumerate((glob('{}\\resulttable-*\\*'.format(CITY_NAME)))):
    print(f'Количество заявок в архиве #{indx}:', pd.read_table(path, sep=';')['appln_id'].nunique())
    df = pd.concat([df, pd.read_table(path, sep=';')])

Количество заявок в архиве #0: 5224


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead.
  after removing the cwd from sys.path.


In [4]:
df.head()

,appln_id,appln_filing_year,appln_auth,techn_field_nr,person_id,person_ctry_code,person_name,person_address
0,416789673,2014,EP,1,20775,US,"CREE, INC.","4600 Silicon Drive,Durham, NC 27703"
1,416789673,2014,EP,3,20775,US,"CREE, INC.","4600 Silicon Drive,Durham, NC 27703"
2,419279440,2014,EP,8,20775,US,"CREE, INC.","4600 Silicon Drive,Durham, NC 27703"
3,422592775,2014,EP,8,20775,US,"CREE, INC.","4600 Silicon Drive,Durham, NC 27703"
4,425994131,2014,EP,8,20775,US,"CREE, INC.","4600 Silicon Drive,Durham, NC 27703"


**Посчитаем количество заявок города по годам**

In [5]:
pd.DataFrame(df.groupby('appln_filing_year').nunique()['appln_id']).T

appln_filing_year,2014,2015,2016,2017
appln_id,1568,1330,1441,885


**Посчитаем топ заявителей по городу**
* Нужно оставить тех, кто входит в первые 2 квартиля

In [6]:
%%time
current_aplicant = []
for indx, person_name in enumerate(df.person_name.unique()):
    #df[df.person_name == person_name].groupby(['appln_filing_year', 'appln_id']).nunique()['appln_id'].groupby('appln_filing_year').size()
    aux_ = pd.DataFrame(df[df.person_name == person_name].groupby(['appln_filing_year', 'appln_id']).nunique()['appln_id'].groupby('appln_filing_year').size()).T
    #aux_.index = [person_name]
    #aux_ = pd.DataFrame(dict(aux_)).reset_index()
    current_aplicant.append([])
    current_aplicant[indx].append(person_name)
    for year in [2014, 2015, 2016, 2017]:
        try:
            current_aplicant[indx].append(aux_[year][0])
        except KeyError:
            current_aplicant[indx].append(0)

Wall time: 3.06 s


In [7]:
df_applicants_leaders = pd.DataFrame(current_aplicant, columns=['person_name', 2014, 2015, 2016, 2017]).assign(summa2014to16 = lambda x:x[2014]+x[2015]+x[2016])
df_applicants_leaders.columns = df_applicants_leaders.columns.tolist()[0:-1] + ['2014-16']

In [8]:
quantiles = df_applicants_leaders['2014-16'].quantile([0.25,0.5,0.75])
quantiles

0.25    1.0
0.50    1.0
0.75    3.0
Name: 2014-16, dtype: float64

In [9]:
df_to_save = df_applicants_leaders[df_applicants_leaders['2014-16'] >= quantiles[0.5]].sort_values('2014-16', \
                                                    ascending=False)
df_to_save.to_excel(f'{CITY_NAME}\\{CITY_NAME}_лидеры.xlsx', index=False)

In [10]:
df[['appln_id', 'appln_filing_year']].nunique()

appln_id             5224
appln_filing_year       4
dtype: int64

**Посчитаем значения по областям с учетом fractional_count** <a class="anchor" id="second-bullet"></a>

> **структура словаря**:
> ```
>    Область
>        Год i
>            Число заявок за год i с учетом fractional_count
>            
>Пример
>    Область
>        2014
>            Число заявок за год i с учетом fractional_count
>        2015
>            Число заявок за год i с учетом fractional_count
>        2016
>            Число заявок за год i с учетом fractional_count
>        2017
>            Число заявок за год i с учетом fractional_count
>```

In [11]:
'''
    На вход:
        сырая табличка заявок
    На выход: словарь структуры выше, содержащий в себе количество заявок по областям по годам 
            с учетом fractional count
    
    dict_of_tech_fileds_per_city :dict: словарь для счёта по тех.областям
    tech_fileds_column :pd Series: колонка для хранения тех.областей
    appln_year :np array: (фактически: единичной длины) 
    tech_fileds_list :np array: лист со всеми (уникальными) тех.бластями заявки, 
                                длина листа равна количеству областей у заявки 
    fractional_count_per_application :float: число с плавающей точкой, равное доле, 
    которую нужно прибавить к каждой области по итогами анализа текущей заявки
    1/количество тех.областей = dfrac{1}{len(tech_fileds_list)}
'''

# создадим словарь структуры, описанной выше
dict_of_tech_fileds_per_city = defaultdict(lambda: defaultdict(np.float16))

# пройдёмся по всем уникальным заявкам в цикле и возьмём
#    А) Колонку с тех областями для каждой заявки
#    Б) Год каждой заявки
for appln_id in df['appln_id'].unique():
    tech_fileds_column = df[df['appln_id'] == appln_id].techn_field_nr
    appln_year = df[df['appln_id'] == appln_id].appln_filing_year.unique()
    # если внезапно было больше одного года для заявки, залогируем это выводом года. пока такого не было
    if len(appln_year) != 1:       
        print(appln_year)
    tech_fileds_list = tech_fileds_column.unique()
    fractional_count_per_application = 1 / len(tech_fileds_list)
    
    # добавим в словарь к каждой встретившеся в заявке области долю fractional_count по итогам анализа заявки
    for tech_filed in tech_fileds_list:
        dict_of_tech_fileds_per_city[tech_filed][appln_year[0]] += fractional_count_per_application

In [12]:
df_techs = pd.DataFrame(dict_of_tech_fileds_per_city).T.sort_index().replace(np.nan, 0)
df_techs = df_techs.assign(summa2014to16 = lambda x:x[2014]+x[2015]+x[2016])
df_techs.columns = df_techs.columns.tolist()[0:-1] + ['2014-16']
df_techs.head()

,2014,2015,2016,2017,2014-16
1,160.683333,91.333333,71.333333,65.633333,323.350000
2,22.916667,21.476190,17.033333,11.116667,61.426190
3,30.333333,22.333333,22.033333,19.983333,74.700000
4,122.533333,113.916667,126.866667,84.250000,363.316667
5,7.333333,3.500000,4.000000,5.416667,14.833333


In [13]:
df_techs

,2014,2015,2016,2017,2014-16
1,160.683333,91.333333,71.333333,65.633333,323.350000
2,22.916667,21.476190,17.033333,11.116667,61.426190
3,30.333333,22.333333,22.033333,19.983333,74.700000
4,122.533333,113.916667,126.866667,84.250000,363.316667
5,7.333333,3.500000,4.000000,5.416667,14.833333
6,248.166667,265.892857,260.366667,206.150000,774.426190
7,47.616667,27.333333,27.033333,16.550000,101.983333
8,111.333333,50.833333,35.250000,44.083333,197.416667
9,19.033333,10.283333,7.033333,11.041667,36.350000
10,51.466667,45.650000,55.033333,34.283333,152.150000


Получим заявителей по областям с $ИТС >= 1.1$
* Нужно убрать хвосты, равные _менее_ $5$ заявок за $2014-2016$ годы

In [19]:
fields_with_its_greater_or_equal1and1 = '''1
4
6
8
10
12
13
15
16
23'''
fields_with_its_greater_or_equal1and1 = fields_with_its_greater_or_equal1and1.split('\n')
print('string of fileds for sql: (', ', '.join(fields_with_its_greater_or_equal1and1), ')')
fields_with_its_greater_or_equal1and1 = tuple(map(int, fields_with_its_greater_or_equal1and1))

string of fileds for sql: ( 1, 4, 6, 8, 10, 12, 13, 15, 16, 23 )


In [20]:
# создадим словарь структуры, описанной выше
dict_of_leaders_per_tech_fileds = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

for appln_id in df['appln_id'].unique():
    
    current_app = df[df['appln_id'] == appln_id][['appln_filing_year', 'techn_field_nr', 'person_name']].drop_duplicates()
#     curr_app_year = current_app.appln_filing_year.unique()
#     curr_app_fields = current_app.techn_field_nr.unique()
    
    for row in current_app.values:
        curr_app_year, tech_filed, person_name = row
        if tech_filed in fields_with_its_greater_or_equal1and1:
            dict_of_leaders_per_tech_fileds[tech_filed][curr_app_year][person_name] += 1


In [21]:
df_leaders_per_tech_fileds = pd.DataFrame([])
flatten_of_leaders_per_tech_fileds = []
for key, val in dict_of_leaders_per_tech_fileds.items():
    cur_field = pd.DataFrame(val).replace(np.nan, 0)
    cur_field = cur_field.reindex(columns=[2014, 2015, 2016, 2017])
    cur_field = cur_field.assign(summa2014to16 = lambda x:x[2014]+x[2015]+x[2016])
    cur_field.columns = cur_field.columns.tolist()[0:-1] + ['2014-16']
    cur_field = cur_field[cur_field['2014-16'] >= 5]
    cur_field['tech_field'] = key
    cur_field.head()
    df_leaders_per_tech_fileds = pd.concat([df_leaders_per_tech_fileds, cur_field])

In [22]:
df_leaders_per_tech_fileds = df_leaders_per_tech_fileds.sort_values(['tech_field', '2014-16'], ascending=[True, False])
df_leaders_per_tech_fileds

,2014,2015,2016,2017,2014-16,tech_field
"Cree, Inc.",131.0,73.0,43.0,50.0,247.0,1
"CREE, INC.",31.0,3.0,11.0,4.0,45.0,1
"Causam Energy, Inc.",20.0,10.0,6.0,17.0,36.0,1
The University of North Carolina at Chapel Hill,3.0,6.0,8.0,6.0,17.0,1
ABB Inc.,2.0,2.0,4.0,1.0,8.0,1
North Carolina State University,5.0,0.0,2.0,4.0,7.0,1
"Valencell, Inc.",5.0,2.0,0.0,0.0,7.0,1
Duke University,2.0,2.0,1.0,1.0,5.0,1
"Protochips, Inc.",3.0,1.0,1.0,0.0,5.0,1
"Red Hat, Inc.",66.0,52.0,78.0,49.0,196.0,4


In [23]:
df_leaders_per_tech_fileds.reset_index().to_excel(f'{CITY_NAME}\\{CITY_NAME}_заявители_по_областям.xlsx', index=False)